In [2]:
import pandas as pd
import numpy as np

# import dtale

In [3]:
products = pd.read_csv('data/BaSalam.products.csv', low_memory=False)

In [4]:
# d = dtale.show(products)
# d.open_browser()

In [5]:
def outliers_number(data: pd.Series, iqr=True,  modified=False, threshold=3, threshold_modified=4.5):
    if iqr:
        q1 = data.quantile(0.25)
        q3 = data.quantile(0.75)
        IQR = q3-q1

        return data[(data > q3+IQR*1.5) | (data < q1-IQR*1.5)].index
    else:
        if modified:
            median = data.median()
            mad = np.median(np.abs(data - median))
            
            modified_z_scores = 0.6745 * (data - median) / mad

            return data[np.abs(modified_z_scores) > threshold_modified].index
        else:
            z_score = (data - data.mean()) / data.std()
            return data[np.abs(data) > threshold].index

In [6]:
cols = products.columns.tolist()

In [7]:
miss_values = pd.Series([int(products[col].isna().sum()) for col in cols])

In [8]:
def outlier_len(iqr, modified):
    outlier_iqr = []
    for col in cols:
        if (type(products[col][0]) in [np.int64, np.float64]):
            outlier_iqr.append(len(outliers_number(products[col], iqr, modified)))
        else:
            outlier_iqr.append(np.nan)

    return pd.Series(outlier_iqr)

In [9]:
pd.DataFrame(index=products.columns.tolist(),
             data={
            'miss value': miss_values.values,
            "type": products.dtypes.values,
            'outlier_iqr': outlier_len(True, False).values,
            'outlier_Z-score': outlier_len(False, False).values,
            'outlier_Z-score modified': outlier_len(False, True).values,
            'Most repeated': [products[col].value_counts().index[0] for col in cols]
            })

,miss value,type,outlier_iqr,outlier_Z-score,outlier_Z-score modified,Most repeated
_id,0,int64,0.0,2411358.0,0.0,13189182
_score,0,float64,76942.0,1024592.0,1024592.0,0.0
sales_count_week,0,int64,76942.0,2703.0,76942.0,0
name,8,object,NaN,NaN,NaN,هفت سین
price,2,float64,329395.0,2411356.0,0.0,1500000.0
status_id,2,float64,2583.0,2411356.0,0.0,2976.0
status_title,2585,object,NaN,NaN,NaN,در دسترس
stock,2,float64,419092.0,1558952.0,0.0,1.0
photo_MEDIUM,540,object,NaN,NaN,NaN,https://statics.basalam.com/public/users/xn5O8...
photo_SMALL,540,object,NaN,NaN,NaN,https://statics.basalam.com/public/users/xn5O8...


In [ ]:
drop_cols =['video_ORIGINAL', 'status_id',
            'photo_MEDIUM', 'photo_SMALL', 'rating_signals',
            'categoryId','vendor_owner_id', 'vendor_status_id',
            'published', 'promotions', 'mainAttribute',
            'vendor_statusId', 'vendor_cityId', 'vendor_id',
            'new_categoryId', 
            ]
data = products.drop(columns=drop_cols)

In [14]:
data[data['_id']==1948970]

,_id,_score,sales_count_week,name,price,status_title,stock,rating_average,rating_count,primaryPrice,...,vendor_freeShippingToSameCity,vendor_provinceId,vendor_has_delivery,vendor_score,vendor_status_title,vendor_owner_city,isFreeShipping,IsAvailable,IsSaleable,categoryTitle
132592,1948970,444.44446,7,خرما خاصویی3کیلویی درجه یک محصول 1402با تخفیف...,2050000.0,در دسترس,35.0,4.6,1333.0,2400000.0,...,NaN,19.0,True,1.0,فعال,قیر,False,True,True,خرما


In [13]:
x = list(data['categoryId'].value_counts().index)
map_ = {}
for i,j in enumerate(x):
    map_[j] = i
categrory_normald = data.copy()
categrory_normald['categoryId'] = categrory_normald['categoryId'].map(map_)

In [14]:
sales_by_catrgory = categrory_normald[categrory_normald['sales_count_week']!=0].groupby('categoryId')['sales_count_week'].mean()
sales_by_catrgory= pd.DataFrame({'category': sales_by_catrgory.index,
              'values': sales_by_catrgory.values})

In [60]:
sales_by_catrgory.sort_values(by='values', ascending=False)[0:10]

,category,values
184,185.0,3.636364
226,229.0,2.800000
264,275.0,2.800000
185,186.0,2.500000
251,260.0,2.500000
165,166.0,2.444444
13,13.0,2.145530
124,125.0,2.090909
28,28.0,2.009901
254,263.0,2.000000


In [22]:
x = data.sort_values(by='price', ascending=False).iloc[:1000][['name', 'price']]
x[x['price'] < 899999990.0]

,name,price
196694,برنج طارم هاشمی درجه 1 عمده(2تنی),899990000.0
1356721,یخچال ساید بای ساید ال جی,899000000.0
863899,فرش دستباف جفت 9 متری خطیبی,899000000.0
1356770,یخچال فریزر 36 فوت ساید بای ساید دوو مدل 3640M...,898940000.0
1339993,چرخ خیاطی 7-8 فرش دوز تیپیکال مدل TW7-6B,898500000.0
...,...,...
1069516,یخچال فریزر دوقلو 40 فوت دوو مدل 2030ss رنگ نق...,689860000.0
1083575,وان و جکوزی ترمو جکوزی دونفره با متعلقات طلایی...,689000000.0
891578,سرویس خواب دونفره نئوکلاسیک سوفیا (ارسال با با...,689000000.0
1356934,یخچال فریزر ساید بای ساید کندی SBS-SCR 34 S سی...,689000000.0


In [33]:
filtered = data[data['categoryTitle'] != 'سایر'].groupby('categoryTitle')[['rating_average', 'categoryTitle']].filter(lambda x : len(x) > 9000)
avg_category_rate = filtered.groupby('categoryTitle')['rating_average'].mean()

In [34]:
avg_category_rate.sort_values(ascending=False)

categoryTitle
کتاب چاپی                        4.272205
ادویه                            4.264962
عطر و ادکلن زنانه و مردانه       3.820212
ظروف سرو و پذیرایی               3.785402
لوازم پخت و پز                   3.760190
گیاهان دارویی                    3.562719
ظروف آشپزخانه                    3.427734
داروهای گیاهی                    3.215669
گیاهان آپارتمانی                 2.476343
لوازم خیاطی                      2.424854
بذر و تخم گیاهان                 2.417628
ابزار قنادی                      2.246346
فلاسک، کلمن و ماگ                2.107724
لوازم تهیه و سرو چای             1.911071
قاشق، چنگال و کارد               1.799050
لامپ، چراغ و مهتابی              1.756268
خاک، کود و آفت کش                1.652326
سایر لوازم یدکی موتورسیکلت       1.575788
روتختی و پتو                     1.529460
محصولات بافتنی                   1.520882
لوازم دستشویی                    1.420168
پارچ، لیوان و بطری               1.417661
سطل، سبد و آبکش                  1.403513
سنگ های قیمتی       

In [41]:
data[data['categoryTitle'] != 'سایر'
        ].groupby('categoryTitle')[[ 'categoryTitle', 'rating_count']
        ].filter(lambda x : len(x) > 9000 and sum(x['rating_count'])> 1000
        ).groupby('categoryTitle')['rating_count'].mean(            
        ).sort_values(ascending=False)

categoryTitle
ادویه                            14.672708
کتاب چاپی                         8.770501
داروهای گیاهی                     5.626436
عطر و ادکلن زنانه و مردانه        5.042523
گیاهان دارویی                     4.709359
لوازم پخت و پز                    4.133074
ظروف سرو و پذیرایی                3.224931
ظروف آشپزخانه                     3.016614
بذر و تخم گیاهان                  2.678167
گیاهان آپارتمانی                  2.235639
لامپ، چراغ و مهتابی               2.040504
لوازم خیاطی                       1.841032
فلاسک، کلمن و ماگ                 1.616573
ابزار قنادی                       1.577346
قاشق، چنگال و کارد                1.551129
لوازم تهیه و سرو چای              1.482495
لوازم حمام                        1.410002
سطل، سبد و آبکش                   1.337179
خاک، کود و آفت کش                 1.244906
پارچ، لیوان و بطری                1.232898
روتختی و پتو                      1.196123
چای ساز و قهوه ساز                1.105083
لوازم دستشویی                     1.0904